# OpenAI Function Calling


<img src='./images/fig_01_01.png' width=800>
- 몇 달 전에 OpenAI API에 추가된 새로운 기능입니다.
- 이 기능을 정확히 어떻게 사용하는지, 그리고 최고의 결과를 얻기 위한 몇 가지 팁과 요령에 대해 알아보겠습니다. 
- OpenAI는 함수 호출을 위한 추가 매개변수를 수락하도록 최신 모델 중 일부를 미세 조정했습니다.
- 이 모델들은 함수 호출이 관련이 있는지 여부를 결정하기 위해 미세 조정되었습니다.

In [1]:
import os

- 이 강의에서는 OpenAI SDK를 직접 사용할 것입니다.
- 이 예제를 진행하기 위해 언어 모델에 제공하기에 흥미로운 함수가 있다고 가정해 보겠습니다.
- 나중에 흥미로운 것이 무엇을 의미하는지 설명하겠지만, 이 새로운 매개변수에는 다양한 사용 사례가 있습니다.
- 여기에서 getCurrentWeather 함수를 정의할 것입니다.
- 이는 OpenAI가 이 기능을 처음 출시할 때 제공한 예제입니다.
- 현재 날씨를 얻는 것은 언어 모델이 자체적으로 할 수 없는 작업이기 때문에 사용하기 좋은 예제입니다.
- 따라서 언어 모델을 함수에 연결하고 싶을 때가 많습니다.

In [2]:
import json

# 예시 더미 함수는 항상 동일한 날씨 정보를 반환하도록 하드코딩
# 실제 환경에서는 백엔드 API 또는 외부 API를 호출할 수 있습니다.
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

- 이 예제에서는 반환되는 정보를 하드 코딩하지만, 실제 환경에서는 날씨 API 또는 외부 지식 소스를 사용할 수 있습니다.
- OpenAI는 함수 정의 목록을 전달할 수 있는 함수라는 새로운 매개변수를 노출했습니다.
- 위의 전체 함수 정의는 바로 여기에 있습니다. 보시다시피 목록이고 목록에 있는 요소는 하나뿐이며, 여기에는 몇 가지 다른 매개변수를 가진 JSON 객체가 있습니다.

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [5]:
from openai import OpenAI

- OpenAI SDK를 가져오고 채팅 완료 엔드포인트를 호출합니다.
- 먼저 모델을 지정할 것입니다. 이 기능이 있는 최신 모델을 지정해야 합니다.
- 그런 다음 위에서 정의한 메시지를 전달할 것입니다. 실행해보고 결과를 확인해봅시다.

In [6]:
client = OpenAI()
response = client.chat.completions.create(
      model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

- 전체 응답을 봅시다. 반환된 메시지에는 assistant 역할이 있고, content는 null이며 대신 name 및 arguments라는 두 개의 객체가 있는 함수 호출 매개변수가 있습니다.
- name은 getCurrentWeather입니다. 이는 우리가 전달한 함수의 이름과 동일하며, arguments는 이 JSON blob입니다.

In [7]:
print(response)

ChatCompletion(id='chatcmpl-9dBE12hQLlJPyy62XXxdlXr6ADlLJ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1719124969, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=82, total_tokens=97))


In [8]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

In [9]:
response_message =response.choices[0].message

In [10]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

- 응답 메시지를 자세히 살펴보겠습니다. 다시 content는 비어 있습니다.

In [11]:
response_message.content

- 함수 호출은 이 사전입니다. 함수 호출의 arguments 매개변수 자체는 JSON 사전이므로 JSON.loads를 사용하여 이를 Python 사전으로 로드할 수 있습니다.

In [12]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather')

In [13]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston'}

- 반환된 arguments를 직접 전달할 수 있습니다. 

In [14]:
args = json.loads(response_message.function_call.arguments)

- OpenAI는 함수를 직접 호출하지 않습니다.
- 우리가 직접 해야 합니다. 대신 호출할 함수의 이름과 해당 함수에 전달할 인수를 알려줍니다.
- 또한 이것이 JSON blob을 반환하도록 학습되었지만 엄격히 적용되지는 않는다는 점도 주목할 가치가 있습니다.

In [15]:
get_current_weather(args)

'{"location": {"location": "Boston"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

- 전달한 메시지가 함수와 관련이 없는 경우에는 어떻게 될까요? 한번 시도해보고 어떤 일이 일어나는지 봅시다.
- 이번에는 반환된 메시지에 일반적인 내용이 있고 함수 호출 매개변수가 없는 새 메시지 집합을 만들어 보겠습니다.
- 내부적으로 모델이 함수를 사용할지 여부를 결정하고 있습니다.

In [16]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [17]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [18]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


- 함수를 사용하거나 사용하지 않도록 모델을 강제할 수 있는 추가 매개변수가 있습니다.
- 이를 살펴보겠습니다. 추가 매개변수는 함수 호출 매개변수입니다. 기본적으로 auto로 설정되어 있습니다. 이는 언어 모델이 선택함을 의미합니다. 우리가 지금까지 해온 일입니다.
- 다른 두 가지 모드가 있습니다. 첫 번째 모드에서는 함수를 호출하도록 강제할 수 있습니다. 항상 함수를 반환하고 싶을 때 유용합니다. 나중에 이 수업에서 그 사용 사례를 살펴보겠습니다.
- auto를 사용하고 언어 모델이 무엇을 할지 선택하도록 하므로 여기서 함수 호출이 필요하지 않음을 인식하고 이전과 같이 역할과 내용만 응답하고 있습니다.

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


- 함수 호출에 사용할 수 있는 또 다른 모드는 none입니다.
- 제공된 함수 중 아무 것도 사용하지 않도록 언어 모델을 강제합니다.
- 이 예제에서는 내용이 함수 호출을 필요로 하지 않기 때문에 상관없습니다.
- 그러나 메시지가 여기서 함수를 호출해야 할 때는 어떻게 될까요? getCurrentWeather 함수를 호출해야 합니다.
- 그러나 아래를 보면 여전히 일반적인 역할과 내용이 보입니다. 이는 함수를 호출하지 않도록 강제하고 있기 때문입니다.

In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response.choices[0].message.content)

Please wait a moment while I check the current weather in Boston.


- 함수 호출 매개변수의 마지막 옵션은 함수를 호출하도록 강제하는 것입니다.
- 이를 수행하는 방법을 살펴보겠습니다. 응답을 보면 실제로 함수 호출 객체가 반환되었으며 이름이 getCurrentWeather이고 몇 가지 인수가 포함되어 있습니다.
- 재미로 함수 호출이 필요 없는 메시지를 전달하지만 강제로 호출하도록 하면 어떻게 되는지 봅시다.

In [32]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9dBHoehvG6F2DJhSnqX6Lz2ubGD8y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1719125204, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95))


In [33]:
print(response.choices[0].message.content)

None


- 여기서 San Francisco, California를 생성하고 있습니다. 다시 실행하면 계속해서 San Francisco, California를 호출합니다.

In [34]:
print(response.choices[0].message.function_call.arguments)

{
  "location": "San Francisco, CA"
}


- 마지막으로 몇 가지 사항을 정리하겠습니다.
- 첫째, 함수 자체와 설명이 OpenAI에 전달하는 토큰 사용 한도에 포함됩니다.
- 함수와 함수 호출을 주석 처리하면 프롬프트 토큰 수가 15로 감소하는 것을 볼 수 있습니다.
- 이 특정 예제에서는 함수 정의와 함수 설명이 많은 토큰을 차지하고 있습니다.
- 이는 OpenAI 모델에 토큰 한도가 있기 때문에 중요합니다.
- 따라서 OpenAI에 전달할 메시지를 구성할 때 이를 염두에 두어야 합니다.

In [38]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9dBKcaRgxv6AbtK1dEvVns2WMK7O0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1719125378, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=89, total_tokens=99))


In [39]:
print(response.choices[0].message.function_call.arguments)

{
"location": "Boston, MA"
}


In [40]:
messages.append(response.choices[0].message)

- 마지막으로 이러한 함수 호출 중 일부와 실제로 함수 호출을 수행한 결과를 언어 모델에 다시 전달하는 방법을 살펴보겠습니다.
- 이는 종종 언어 모델을 사용하여 호출할 함수를 결정한 다음 해당 함수를 실행하고 그 결과를 언어 모델에 다시 전달하여 최종 응답을 얻고 싶을 때 중요합니다.

In [41]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)
print(observation)

{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


- 이 메시지를 가져와 메시지 목록에 추가할 것입니다.
- 언어 모델이 제공하는 인수로 getCurrentWeather 함수를 호출하는 것을 시뮬레이션할 수 있습니다.
- 이를 변수에 저장한 다음 방금 호출한 함수의 응답을 나타내는 새 메시지를 목록에 추가할 수 있습니다.

In [42]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

- 새로운 유형의 메시지로 이를 수행합니다. 역할이 함수와 동일합니다.
- 이는 함수 호출의 응답임을 언어 모델에 전달하는 데 사용됩니다. 그런 다음 이름도 전달하는데, 이는 함수의 이름입니다.
- 그리고 위에서 계산한 observation과 동일하게 설정할 수 있는 content 변수를 전달합니다.
- 이 메시지 목록으로 언어 모델을 호출하면 언어 모델이 observation의 응답이 생성됩니다.

In [43]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response.choices[0].message.content)

The current weather in Boston, MA is sunny and windy with a temperature of 72°F.


# LangChain Expression Language (LCEL)

<img src='./images/fig_02_01.png' width=800>

<img src='./images/fig_02_03.png' width=800>

In [1]:
import os
# import openai

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

- 단순 체인을 설정합니다. 

In [12]:
prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 짧은 농담을 말해줘."
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [13]:
chain = prompt | model | output_parser

In [14]:
response = chain.invoke({"topic": "bears"})
print(response)

왜 곰들은 항상 춥죠? 

왜냐하면 그들은 언제나 겨울모드에 있거든요!


## More complex chain

- And Runnable Map to supply user-provided inputs to the prompt.
- 이제 더 복잡한 체인을 만들어 보겠습니다. 여기서는 검색 기능이 추가된 생성을 구현합니다.

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [23]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [24]:
results = retriever.invoke( "where did harrison work?")
print(results)

[Document(page_content='harrison worked at kensho'), Document(page_content='bears like to eat honey')]


In [25]:
results = retriever.invoke("what do bears like to eat?")
print(results)

[Document(page_content='bears like to eat honey'), Document(page_content='harrison worked at kensho')]


In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [27]:
from langchain.schema.runnable import RunnableMap

- RunnableMap을 사용하여 검색된 문맥과 질문을 매핑합니다.

In [28]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [29]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [30]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [31]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [32]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [34]:
runnable = prompt | model

In [35]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9f9a285c-36f9-43ed-9e48-6d41f1bcf27e-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

In [36]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [37]:
model = model.bind(functions=functions)

In [38]:
runnable = prompt | model

In [39]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-600262c2-ed8a-4adc-859c-3647da56f043-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117})

- 패트리어츠(Patriots)는 미국의 프로 미식축구 팀인 뉴잉글랜드 패트리어츠(New England Patriots)를 줄여서 부르는 말입니다. 이 팀은 NFL(National Football League)에 속해 있으며, 매사추세츠주 폭스버러에 기반을 두고 있습니다. 패트리어츠는 NFL에서 매우 성공적인 팀 중 하나로, 여러 차례 슈퍼볼 우승을 차지한 경력이 있습니다.

## Fallbacks
- 이제 에러 처리를 위한 폴백 체인을 설정하겠습니다.
- 첫 번째 모델이 실패할 경우 폴백 모델을 사용합니다.

In [66]:
from langchain.llms import OpenAI # 옛날 모델 
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

- 단순 체인을 설정합니다. 이 체인은 언어 모델의 출력을 JSON으로 디코딩합니다.

In [69]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)

simple_chain = simple_model | json.loads

In [70]:
challenge = "세 개의 시를 JSON 블롭으로 작성하세요. 각 시는 제목, 작가, 첫 줄로 구성된 JSON 블롭입니다."

In [71]:
response = simple_model.invoke(challenge)
print(response)



{
  "title": "봄",
  "author": "김소월",
  "first_line": "봄이 오면 산들도 꽃피고"
}

{
  "title": "가을",
  "author": "윤동주",
  "first_line": "가을이 오면 나는 더 사랑하리라"
}

{
  "title": "겨울",
  "author": "정지용",
  "first_line": "겨울이 오면 나는 더 추억하리라"
}


- json 형식 생성에 실패

In [72]:
type(response)

str

**Note**: The next line is expected to fail.

In [74]:
from langchain_openai import ChatOpenAI # 최근 모델

model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [75]:
response = chain.invoke(challenge)

In [76]:
print(response)

[{'title': '봄', 'author': '윤동주', 'first_line': '봄이 오나 봄이 오나 봄이 오나'}, {'title': '가을', 'author': '김소월', 'first_line': '하늘 높이 구름 없이 맑은 가을하늘'}, {'title': '겨울', 'author': '이상', 'first_line': '눈 내리는 밤이면 눈 내리는 밤이면'}]


- 최근 모델 성공

In [77]:
type(response[0])

dict

- fallbacks 를 쓰면 에러가 나도 원래 chain로 돌아가서 실행을 한다.
- 첫 번째 모델이 실패하면 폴백 모델이 호출됨

In [49]:
final_chain = simple_chain.with_fallbacks([chain])

In [50]:
response = final_chain.invoke(challenge)
print(response)

[{'title': '봄', 'author': '윤동주', 'first_line': '봄이 오나 봄이 오나 봄이 오나'}, {'title': '가을', 'author': '김소월', 'first_line': '하늘 높이 구름 없이 맑은 가을하늘'}, {'title': '겨울', 'author': '이상', 'first_line': '눈 내리는 밤이면 눈 내리는 밤이면'}]


## Interface
- LCEL을 사용하면 다양한 방식으로 런너블을 호출할 수 있습니다. 여기에는 invoke, stream, batch 메서드가 포함됩니다.
  
<img src='./images/fig_02_02.png' width=800>

In [78]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [79]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

- batch 메서드는 여러 입력에 대해 런너블을 호출합니다. 이는 병렬 처리를 통해 성능을 향상시킬 수 있습니다.

In [80]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship any longer!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

- stream 메서드는 스트리밍 방식으로 응답을 반환합니다. 이 방법은 응답을 점진적으로 처리할 수 있습니다.

In [81]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



- 모든 동기 메서드는 비동기 버전도 제공됩니다. 예를 들어 ainvoke, astream, abatch 메서드가 있습니다.

In [82]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears like fast food? Because they can't catch it!"

# OpenAI Function Calling In LangChain

- OpenAI 함수와 LangChain 표현 언어의 사용법을 소개합니다.
- Pydantic 라이브러리를 소개합니다. Pydantic은 OpenAI 함수를 쉽게 만들 수 있게 도와주는 Python 라이브러리입니다.


- Pydantic이란?
    - Pydantic은 Python의 데이터 검증 라이브러리로, 다양한 스키마를 정의하고 이를 JSON으로 내보내기 쉽게 합니다.
    - Pydantic 객체를 사용하여 OpenAI 함수 설명을 작성할 수 있습니다.


<img src='./images/fig_03_01.png' width=800>

- Pydantic 클래스 정의
    - 일반적인 Python 클래스와 유사하며, init 메서드 대신 클래스 정의 아래 속성과 타입을 나열합니다.
    - 이 클래스들은 실제로 사용되지는 않고, OpenAI 함수의 JSON을 생성하기 위해 사용됩니다.


<img src='./images/fig_03_02.png' width=800>

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [1]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

- Pydantic 데이터 클래스는 Python의 데이터 클래스와 Pydantic의 검증 기능이 결합된 것입니다.
- 이들은 데이터 구조를 간결하게 정의하면서 데이터가 지정된 타입과 제약 조건을 준수하도록 보장합니다.
- 표준 Python에서는 다음과 같이 클래스를 생성합니다:

In [2]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [3]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [4]:
foo.name

'Joe'

In [5]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [6]:
foo.age

'bar'

In [7]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [8]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [9]:
foo_p.name

'Jane'

**Note**: The next cell is expected to fail.

In [10]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [20]:
class Class(BaseModel):
    students: List[pUser]

In [21]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [22]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition
- 다음은 Pydantic을 사용하여 OpenAI 함수를 생성하는 코드 예제입니다.
- WeatherSearch 클래스는 Pydantic의 BaseModel을 상속받아 정의됩니다.
- 클래스에는 airport_code라는 문자열 속성이 있으며, 이 속성에는 "날씨를 가져올 공항 코드"라는 설명이 붙어 있습니다.
- 클래스의 docstring은 "Call this with an airport code to get the weather at that airport"로, 이 함수가 특정 공항의 날씨 정보를 가져오는 데 사용된다는 설명을 포함합니다.

In [24]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

- convert_pydantic_to_openai_function 함수를 사용하여 WeatherSearch 클래스를 OpenAI 함수로 변환합니다.

In [25]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

In [26]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

- weather_function 변수는 변환된 OpenAI 함수의 JSON 스키마를 포함합니다.

In [27]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

- description 이 없으면 에러가 나야 하지만

In [37]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

- 지금은 안난다 (구버전은 났음)

In [38]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [39]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [40]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [32]:
from langchain_openai import ChatOpenAI

In [33]:
model = ChatOpenAI()

- model.invoke("what is the weather in SF today?", functions=[weather_function])를 호출하여 모델에게 "SF의 오늘 날씨는?"이라는 질문을 전달합니다.
- functions 인수에 weather_function을 전달하여, 모델이 이 질문에 대해 weather_function 함수를 사용할 수 있도록 합니다.
- weather_function은 이전에 Pydantic을 사용하여 정의한 날씨 검색 함수로, 공항 코드를 사용하여 해당 공항의 날씨 정보를 가져오는 역할을 합니다.

In [34]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-beb9474e-b53d-4a94-90a5-102f49a6102c-0', usage_metadata={'input_tokens': 70, 'output_tokens': 17, 'total_tokens': 87})

- model.bind(functions=[weather_function])를 호출하여 모델에 weather_function을 바인딩합니다.
- 이렇게 하면 모델 인스턴스가 weather_function을 항상 사용할 수 있게 됩니다.
- model_with_function 변수는 함수가 바인딩된 모델 인스턴스를 참조합니다.
- model_with_function.invoke("what is the weather in sf?")를 호출하여 모델에게 "SF의 날씨는?"이라는 질문을 전달합니다.
- 바인딩된 weather_function이 이 질문에 대해 사용됩니다.

In [35]:
model_with_function = model.bind(functions=[weather_function])

In [36]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-46bc28d0-5234-4168-81d6-5517af450401-0', usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86})

## Forcing it to use a function

- `model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})`를 호출하여 모델에 `weather_function`을 바인딩하고, 특정 함수 호출을 강제하도록 설정합니다.
  - `functions=[weather_function]`: 모델에 바인딩할 함수 리스트를 지정합니다.
  - `function_call={"name":"WeatherSearch"}`: 모델이 항상 "WeatherSearch"라는 이름의 함수를 호출하도록 강제합니다.
- `model_with_forced_function` 변수는 함수가 바인딩되고 특정 함수 호출이 강제된 모델 인스턴스를 참조합니다.
- `model_with_forced_function.invoke("what is the weather in sf?")`를 호출하여 모델에게 "SF의 날씨는?"이라는 질문을 전달합니다.
- 이 설정 덕분에 모델은 항상 `WeatherSearch` 함수를 사용하여 질문에 응답하게 됩니다.

In [41]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [42]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fbbbfdf8-e817-4581-908f-e292dcd7ab14-0', usage_metadata={'input_tokens': 79, 'output_tokens': 7, 'total_tokens': 86})

In [43]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1ae737a8-ea56-467e-945b-a6b8de29247a-0', usage_metadata={'input_tokens': 74, 'output_tokens': 7, 'total_tokens': 81})

## Using in a chain


다음은 프롬프트 템플릿과 바인딩된 모델을 사용하여 체인을 호출하는 코드 예제입니다.

- `ChatPromptTemplate.from_messages`를 사용하여 프롬프트 템플릿을 생성합니다.
  - 이 템플릿은 시스템 메시지와 사용자 메시지로 구성됩니다.
  - 시스템 메시지: "You are a helpful assistant" (당신은 도움이 되는 어시스턴트입니다)
  - 사용자 메시지: `{input}` (사용자 입력을 받을 자리 표시자)
- `prompt` 변수는 생성된 프롬프트 템플릿을 참조합니다.
- `prompt | model_with_function`을 사용하여 프롬프트 템플릿과 바인딩된 모델을 체인으로 결합합니다.
  - `|` 연산자는 프롬프트 템플릿의 출력을 모델에 전달하는 체인을 생성합니다.
- `chain.invoke({"input": "what is the weather in sf?"})`를 호출하여 체인에 입력을 전달하고 결과를 가져옵니다.
  - 입력 딕셔너리에는 `input` 키가 있으며, 값으로 "what is the weather in sf?"를 전달합니다.
- 이 체인은 프롬프트 템플릿을 통해 입력을 받아 모델로 전달하고, 모델은 바인딩된 `weather_function`을 사용하여 응답을 생성합니다.

이 예제는 프롬프트 템플릿과 바인딩된 모델을 체인으로 결합하여 사용자 입력을 처리하고 모델의 응답을 가져오는 방법을 보여줍니다.

In [44]:
from langchain.prompts import ChatPromptTemplate

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [46]:
chain = prompt | model_with_function

In [47]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 75, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-808b6bad-a7fe-44eb-b0b9-a3df9396ec09-0', usage_metadata={'input_tokens': 75, 'output_tokens': 17, 'total_tokens': 92})

## Using multiple functions

다음은 여러 Pydantic 함수를 바인딩하고 호출하는 OpenAI 모델 코드 예제입니다.

- `ArtistSearch` 클래스는 Pydantic의 `BaseModel`을 상속받아 정의됩니다.
  - `artist_name` 속성은 조회할 아티스트의 이름을 나타내며, 문자열로 타입이 지정되어 있습니다.
  - `n` 속성은 결과의 개수를 나타내며, 정수로 타입이 지정되어 있습니다.
  - 클래스의 docstring은 "Call this to get the names of songs by a particular artist"로, 특정 아티스트의 노래 목록을 가져오는 데 사용된다는 설명을 포함합니다.
- `functions` 리스트에는 `WeatherSearch`와 `ArtistSearch` 클래스에서 변환된 OpenAI 함수가 포함됩니다.
- `model.bind(functions=functions)`를 호출하여 모델에 여러 함수를 바인딩합니다.
- `model_with_functions.invoke("what is the weather in sf?")`를 호출하여 모델에게 "SF의 날씨는?"이라는 질문을 전달합니다. 모델은 바인딩된 `weather_function`을 사용하여 응답합니다.
- `model_with_functions.invoke("what are three songs by taylor swift?")`를 호출하여 모델에게 "Taylor Swift의 노래 세 곡은 무엇인가요?"라는 질문을 전달합니다. 모델은 바인딩된 `artist_function`을 사용하여 응답합니다.
- `model_with_functions.invoke("hi!")`를 호출하여 모델에게 "안녕하세요!"라는 인사말을 전달합니다. 이 경우, 모델은 함수 호출 없이 일반적인 응답을 제공합니다.

이 예제는 여러 Pydantic 함수를 바인딩하여 다양한 질문에 대해 적절한 함수를 선택하여 응답하는 방법을 보여줍니다. 모델은 질문의 내용에 따라 적절한 함수를 호출하거나, 필요하지 않은 경우 일반적인 응답을 제공할 수 있습니다.

In [48]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [49]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [50]:
model_with_functions = model.bind(functions=functions)

In [51]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-597a4b35-9247-467a-8fcf-56df449a87ee-0', usage_metadata={'input_tokens': 116, 'output_tokens': 17, 'total_tokens': 133})

In [52]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-dc0befe1-cd4b-4d85-b1b1-ec61ef81a792-0', usage_metadata={'input_tokens': 118, 'output_tokens': 21, 'total_tokens': 139})

In [53]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1dc3fbf7-9525-49fc-95af-ac152c36b433-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121})

# Tagging and Extraction Using OpenAI functions

<img src="./images/fig_04_01.png" widht=800>

### 태깅 (Tagging)
1. **개요**: 비구조화된 텍스트에 구조화된 설명을 입력하여 구조화된 데이터를 생성.
2. **예시**: 텍스트의 감정과 언어 태그를 추출.
3. **과정**:
   - 비구조화된 텍스트와 구조화된 설명을 입력.
   - LLM을 사용하여 텍스트의 감정과 언어를 태그로 추출.
   - 구조화된 설명에 따라 결과 생성.

In [10]:
# import os
# import openai

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

1. **설정**:
   - 필요한 함수와 클래스 임포트 (typing, Pydantic 등).
   - Pydantic 모델 생성 (태깅에 사용).
   - 'convert_pydantic_to_openai_function' 메소드 사용.

In [5]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

2. **태깅 기능 구현**:
   - 프로토타입 태깅 모델 생성.
   - 감정과 언어 태그를 위한 구조화된 설명 추가.
   - 모델에 프롬프트와 언어 모델 연결.
   - 사용자 입력을 받아 태깅 함수 호출.
   - 결과: 감정(positive), 언어(English) 등 추출.

In [6]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [7]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [12]:
model = ChatOpenAI(temperature=0)

In [13]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [15]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [16]:
tagging_chain = prompt | model_with_functions

In [17]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c7883300-4dfe-4628-b030-66da464d0619-0', usage_metadata={'input_tokens': 108, 'output_tokens': 10, 'total_tokens': 118})

In [18]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a867821-3ad7-453d-82d2-e37edcd948ec-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121})

In [19]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [20]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [23]:
tagging_chain.invoke({"input": "이 순대 존맛탱이네."})

{'sentiment': 'pos', 'language': 'ko'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

<img src="./images/fig_04_02.png" widht=800>

### 추출 (Extraction)
1. **개요**: 텍스트에서 특정 엔티티를 추출.
2. **차이점**: 태깅과 달리 리스트 형태로 엔티티를 추출.
3. **예시**: 기사에서 언급된 논문 리스트 추출.

3. **추출 기능 구현**:
   - 다수의 정보를 추출하기 위해 클래스 정의.
   - 예: 사람 정보(person schema) 추출.
   - 추출한 정보를 리스트로 관리.

In [24]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [25]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [26]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'$defs': {'Person': {'description': 'Information about a person.',
    'properties': {'name': {'description': "person's name", 'type': 'string'},
     'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
      'description': "person's age"}},
    'required': ['name', 'age'],
    'type': 'object'}},
  'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [27]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [28]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a0763e4d-88cc-49b7-9fce-4f30971f659d-0', usage_metadata={'input_tokens': 95, 'output_tokens': 21, 'total_tokens': 116})

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [30]:
extraction_chain = prompt | extraction_model

In [31]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 112, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f941acd5-4354-4965-8ef4-413f2cbd3072-0', usage_metadata={'input_tokens': 112, 'output_tokens': 21, 'total_tokens': 133})

In [32]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [33]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

### 출력 파서 (Output Parser)
1. **JSON 출력 파서**: JSON 형식으로 출력된 데이터를 파싱하여 활용.
2. **LangChain**: JSON 출력 파서 사용으로 편리한 데이터 활용.


In [34]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [35]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [36]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]

## 실제 적용 예시

우리는 태깅을 더 큰 텍스트에 적용할 수 있습니다.

예를 들어, 이 블로그 포스트를 로드하고 텍스트의 일부에서 태그 정보를 추출해 봅시다.

In [37]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [39]:
doc = documents[0]

In [40]:
page_content = doc.page_content[:10000]

In [41]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [49]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [50]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [51]:
tagging_chain.invoke({"input": page_content})

{'summary': 'Building autonomous agents powered by LLM (large language model) with components like planning, memory, and tool use. Includes task decomposition, self-reflection, and challenges.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, challenges'}

In [52]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [53]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [54]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [55]:
# template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

# Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

# Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

template = '''
기사가 제공됩니다. 이 기사에서 언급된 모든 논문을 추출하세요.

기사의 제목은 추출하지 마세요. 논문이 언급되지 않았다면, 빈 목록을 반환하세요.

추가적인 정보를 추측하거나 만들어내지 말고, 텍스트에 정확히 있는 내용만 추출하세요.
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [56]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [57]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': None},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': None},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': None},
 {'title': 'ReAct (Yao et al. 2023)', 'author': None},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': None},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)', 'author': None},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)', 'author': None}]

In [58]:
extraction_chain.invoke({"input": "hi"})

[]

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [60]:
splits = text_splitter.split_text(doc.page_content)

In [61]:
len(splits)

15

In [62]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [63]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [64]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general probl

In [66]:
from langchain.schema.runnable import RunnableLambda

In [67]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [68]:
prep.invoke("hi")

[{'input': 'hi'}]

In [71]:
chain = prep | extraction_chain.map() | flatten

In [72]:
response = chain.invoke(doc.page_content)

In [73]:
response

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'},
 {'title': 'Algorithm Distillation for In-Context Reinforcement Learning',
  'author': 'Laskin et al. 2023'},
 {'title': 'LSH Forest: Self-Tuning Indexes for Similarity Search',
  'author': 'Marius Muja and David G. Lowe'},
 {'title': 'ANNOY: Approximate Nearest Neighbors Oh Yeah',
  'author': 'Erik Bernhardsson'},
 {'title': 'Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs',
  'author': 'Yu. A. Malkov and D. A. Yashunin'},
 {'title': 'FAISS: A 